In [2]:
#imports
import torch
from torch import nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch
from torch.utils.data import TensorDataset
import ast
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from sklearn import metrics
from reg_sklearn import DataHandler
from econml.dml import LinearDML


import matplotlib.pyplot as plt


In [3]:
##load the data
patients = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/patients.csv')
organs = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/organs.csv')
outcomes = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/outcomes.csv')
outcomes_noiseless = pd.read_csv('C:/Users/Ernesto/OneDrive - ETH Zurich/Desktop/MT/COMET/synthetic_data_generation/outcomes_noiseless.csv')

outcome = 'eGFR'



outcomes = outcomes[['pat_id', 'org_id', outcome]]
outcomes_noiseless = outcomes_noiseless[['pat_id', 'org_id', outcome]]

outcomes = outcomes.dropna()
outcomes_noiseless = outcomes_noiseless.dropna()

outcomes = outcomes.iloc[:, 2]
outcomes = outcomes.to_numpy()

# CAUTION: Uncomment this line if loading data from a csv file
# self.outcomes = self.outcomes.map(ast.literal_eval)
# self.outcomes_noiseless = self.outcomes_noiseless.map(ast.literal_eval)


# if outcome == 'eGFR_3':
#     outcomes = self.outcomes.map(lambda x: x['eGFR'][2] if x and 'eGFR' in x else None)
#     outcomes_noiseless = self.outcomes_noiseless.map(lambda x: x['eGFR'][2] if x and 'eGFR' in x else None)

# else:
#     raise ValueError('Outcome not supported')


patients = pd.get_dummies(patients)
# organs = pd.get_dummies(organs) 

In [4]:
#cluster the organs treatment into 10 clusters
from kmodes.kprototypes import KPrototypes

# Assuming 'df' is your DataFrame
# Specify the columns of your categorical features
catColumnsPos = [organs.columns.get_loc(col) for col in list(organs.select_dtypes('object').columns)]

kproto = KPrototypes(n_clusters=10, init='Cao', verbose=2)
clusters = kproto.fit_predict(organs, categorical=catColumnsPos)




Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 230, ncost: 1177181.139282824
Run: 1, iteration: 2/100, moves: 97, ncost: 1082506.871248864
Run: 1, iteration: 3/100, moves: 46, ncost: 1051069.555367863
Run: 1, iteration: 4/100, moves: 42, ncost: 1030857.8971465278
Run: 1, iteration: 5/100, moves: 27, ncost: 1021194.1423270683
Run: 1, iteration: 6/100, moves: 24, ncost: 1014490.2723338227
Run: 1, iteration: 7/100, moves: 17, ncost: 1010465.7560091338
Run: 1, iteration: 8/100, moves: 8, ncost: 1008809.4881569692
Run: 1, iteration: 9/100, moves: 11, ncost: 1006880.3039079453
Run: 1, iteration: 10/100, moves: 13, ncost: 1004661.8920488937
Run: 1, iteration: 11/100, moves: 8, ncost: 1003687.418274928
Run: 1, iteration: 12/100, moves: 6, ncost: 1002629.8463167349
Run: 1, iteration: 13/100, moves: 10, ncost: 1001211.9397137293
Run: 1, iteration: 14/100, 

In [5]:

%matplotlib notebook
%pip install plotly
!pip install --upgrade nbformat

from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

pca = PCA(n_components=3)
organs = pd.get_dummies(organs)
pca_result = pca.fit_transform(organs)

import plotly.express as px

fig = px.scatter_3d(
    pca_result, x=0, y=1, z=2, color=clusters,
    labels={'0': 'PC1', '1': 'PC2', '2': 'PC3'}
)

fig.show()

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#normalize the patients data in order for logistic regression ot converge better

scaler = StandardScaler()
patients = scaler.fit_transform(patients)


In [14]:
#fit the model
from econml.dml import LinearDML
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier



first_stage = lambda: GridSearchCV(
                estimator=RandomForestClassifier(),
                param_grid={
                        'max_depth': [3, None],
                        'n_estimators': (10, 30, 50, 100, 200),
                        'max_features': (1,2,3)
                    }, n_jobs=-1, scoring='accuracy'
                )
second_stage = lambda: GridSearchCV(
                estimator=RandomForestRegressor(),
                param_grid={
                        'max_depth': [3, None],
                        'n_estimators': (10, 30, 50, 100, 200),
                        'max_features': (1,2,3)
                    }, n_jobs=-1, scoring='neg_mean_squared_error'
                )

est = LinearDML(model_t='nnet', model_y='nnet', discrete_treatment=True)
est.fit(outcomes, clusters , X=patients, W=None)
point = est.const_marginal_effect(X=patients)
effect = est.effect(patients, T0=0, T1=1)

c:\Users\Ernesto\anaconda3\envs\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

c:\Users\Ernesto\anaconda3\envs\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

c:\Users\Ernesto\anaconda3\envs\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

c:\Users\Ernesto\anaconda3\envs\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

c:\Users\Ernesto\anaconda3\envs\env\Lib\site-packages\sklearn\neural_network\_multilayer